In [1]:
import os
import math
import torch
from torch import nn, optim
from torchvision import transforms
from torch.utils.data import DataLoader
from PIL import Image
from torchvision import models
from analyze import XRayDataset, load_model
from collections import defaultdict

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

dataset, loader, model = load_model(device)
num_classes = len(dataset.classes)

criterion = nn.CrossEntropyLoss(weight=dataset.class_weights.to(device))

images_per_class = defaultdict(lambda: [])

j = 0
count = len(dataset)
for inputs, labels, filenames in loader:
    with torch.set_grad_enabled(False):
        j += 1
        if j % 100 == 0:
            print(f'{j*10}/{count}')
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)

        _, preds = torch.max(outputs, 1)

        for i in range(len(labels)):
            label = labels[i:i+1]
            pred = preds[i]
            output = outputs[i:i+1]
            loss = criterion(output, label)
            
            images_per_class[labels[i].item()].append({
                'image': filenames[i],
                'correct': bool(label == pred),
                'loss': loss.item()
            })

1000/7992
2000/7992
3000/7992
4000/7992
5000/7992
6000/7992
7000/7992
8000/7992


In [6]:
import json
best_ten = {}

for label, images in images_per_class.items():
    images = filter(lambda i: i['correct'], images)
    images = sorted(images, key=lambda i: i['loss'])
    best_ten[dataset.classes[label]] = images[:10]

with open('best10.json', 'w') as f:
    json.dump(best_ten, f, indent=2)